# Set-up

In [1]:
import os
import sys
import yaml
import logging
import mudata
import pandas as pd

# Change path to wherever you have repo locally
sys.path.append('/oak/stanford/groups/engreitz/Users/ymo/Tools/cNMF_benchmarking/cNMF_benchmakring_pipeline')

from Evaluation.src import (
    compute_categorical_association,
    compute_geneset_enrichment,
    compute_trait_enrichment,
    compute_perturbation_association,
    compute_explained_variance_ratio,
    compute_motif_enrichment
)

from Evaluation.src.enrichment_trait import process_enrichment_data

/home/users/ymo/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# I/O paths
path_config = "/oak/stanford/groups/engreitz/Users/ymo/Tools/gene_network_evaluation/examples/evaluation/evaluation_pipeline.yml"
config = yaml.safe_load(open(path_config))

## I/O

In [3]:
io_config = config['io']
io_config

{'path_mdata': '/oak/stanford/groups/engreitz/Users/ymo/NMF_re-inplementing/Results/cNMF_evaluation_7.31.25/output/consensus_NMF/cNMF_300_2.0.h5mu',
 'path_out': '/oak/stanford/groups/engreitz/Users/ymo/NMF_re-inplementing/Results/cNMF_evaluation_7.31.25/output/300',
 'data_key': 'rna',
 'prog_key': 'cNMF'}

In [4]:
# Load mdata
path_mdata = io_config['path_mdata']
mdata = mudata.read(path_mdata)
mdata

MuData object with n_obs × n_vars = 92284 × 5751
  2 modalities
    rna:	92284 x 5451
      obs:	'sample', 'species', 'gene_count', 'tscp_count', 'mread_count', 'leiden'
      obsm:	'X_pca', 'X_umap'
      layers:	'norm10k'
    cNMF:	92284 x 300
      obs:	'sample', 'species', 'gene_count', 'tscp_count', 'mread_count', 'leiden'
      uns:	'var_names'
      varm:	'loadings'

In [5]:
mdata

MuData object with n_obs × n_vars = 92284 × 5751
  2 modalities
    rna:	92284 x 5451
      obs:	'sample', 'species', 'gene_count', 'tscp_count', 'mread_count', 'leiden'
      obsm:	'X_pca', 'X_umap'
      layers:	'norm10k'
    cNMF:	92284 x 300
      obs:	'sample', 'species', 'gene_count', 'tscp_count', 'mread_count', 'leiden'
      uns:	'var_names'
      varm:	'loadings'

In [6]:
# Make path_out directory
path_out = io_config['path_out']
if not os.path.exists(path_out):
    os.makedirs(path_out)

In [7]:
# Update cNMF key to cNMF_30 and save
old_prog_key = io_config['prog_key']
prog_key = os.path.basename(path_out)
mdata.mod[prog_key] = mdata.mod.pop(old_prog_key)
mdata.update()
mdata.write(os.path.join(path_out, f'{prog_key}.h5mu'))

In [8]:
# Get data key
data_key= io_config['data_key']
data_key

'rna'

## Run categorical association

In [21]:
# Run categorical association and save results
categorical_assocation_config = config['categorical_association']
categorical_keys = categorical_assocation_config['categorical_keys']
for key in categorical_keys:
    results_df, posthoc_df = compute_categorical_association(
        mdata, 
        prog_key=prog_key,
        categorical_key=key,
        **categorical_assocation_config,
    )
    results_df.to_csv(os.path.join(path_out, f'{prog_key}_{key}_association_results.txt'), sep='\t', index=False) 
    posthoc_df.to_csv(os.path.join(path_out, f'{prog_key}_{key}_association_posthoc.txt'), sep='\t', index=False)

INFO:root:Performing tests at single-cell level. Significance will likely be inflated
Testing sample association: 100%|██████████| 300/300 [00:36<00:00,  8.16programs/s]
INFO:root:Running jamboree specific version of posthoc with pearsonr, this is not yet integrated into the main pipeline
Identifying differential sample: 100%|██████████| 300/300 [01:21<00:00,  3.69programs/s]


## Run perturbation association

In [10]:
perturbation_assocation_config = config['perturbation_association']
perturbation_assocation_config

{'groupby_key': 'sample',
 'collapse_targets': True,
 'pseudobulk': False,
 'reference_targets': ['non-targeting'],
 'n_jobs': -1,
 'inplace': False}

In [13]:
# Run perturbation association
target_type = "gene" if perturbation_assocation_config["collapse_targets"] else "guide"
if perturbation_assocation_config["groupby_key"] is not None:
    logging.info("groupby_key provided, running perturbation association on each group")
    groupby_key = perturbation_assocation_config.pop("groupby_key")
    data_key = io_config["data_key"]
    perturbation_assocation_df = pd.DataFrame()
    for group in mdata[data_key].obs[groupby_key].unique():
        mdata_ = mdata[mdata[data_key].obs[groupby_key] == group]
        test_stats_df = compute_perturbation_association(
            mdata_, 
            prog_key=prog_key,
           # data_key=data_key,
            **perturbation_assocation_config,
        )
        test_stats_df.to_csv(os.path.join(path_out, f'{prog_key}_{target_type}_{groupby_key}_{group}_perturbation_association.txt'), sep='\t', index=False)
else:
    logging.info("No groupby_key provided, running perturbation association on full dataset")
    perturbation_assocation_config.pop("groupby_key")
    perturbation_assocation_df = compute_perturbation_association(
        mdata, 
        prog_key=prog_key,
        data_key=data_key,
        **perturbation_assocation_config,
    )
    perturbation_assocation_df.to_csv(os.path.join(path_out, f'{prog_key}_{target_type}_perturbation_association.txt'), sep='\t', index=False)

KeyError: 'groupby_key'

## Run gene set enrichment analysis

In [9]:
# Gene-set enrichment
gene_set_enrichment_config = config['gene_set_enrichment']
libraries = gene_set_enrichment_config.pop('libraries')
gene_set_enrichment_df = pd.DataFrame()
data_key = io_config['data_key']
for library in libraries:
    logging.info(f'Running gene-set enrichment analysis for {library}')
    pre_res = compute_geneset_enrichment(
        mdata, 
        prog_key=prog_key,
        data_key=data_key,
        library=library,
        **gene_set_enrichment_config,
    )

    if gene_set_enrichment_config["method"] == "fisher":
        pre_res = pre_res.rename(columns={"Term": "term", "P-value": "pval", "Adjusted P-value": "adj_pval", "Odds Ratio": "enrichment", "Genes": "genes"})
    elif gene_set_enrichment_config["method"] == "gsea":
        pre_res = pre_res.rename(columns={"Term": "term", "NOM p-val": "pval", "FDR q-val": "adj_pval", "NES": "enrichment", "Lead_genes": "genes"})
    
    # Save results
    pre_res.to_csv(os.path.join(path_out, f'{prog_key}_{library}_{gene_set_enrichment_config["method"]}_geneset_enrichment.txt'), sep='\t', index=False)

INFO:root:Running gene-set enrichment analysis for Reactome_2022


INFO:root:Downloading and generating Enrichr library gene sets...
INFO:root:0031 gene_sets have been filtered out when max_size=500 and min_size=5
Running Fisher enrichment: 100%|██████████| 300/300 [01:13<00:00,  4.05programs/s]
INFO:root:Running gene-set enrichment analysis for GO_Biological_Process_2023
INFO:root:Downloading and generating Enrichr library gene sets...
INFO:root:0014 gene_sets have been filtered out when max_size=500 and min_size=5
Running Fisher enrichment: 100%|██████████| 300/300 [03:19<00:00,  1.51programs/s]


## Run trait enrichment analysis

In [23]:
# Run trait enrichment, process and save results
trait_enrichment_config = config['trait_enrichment']
pre_res_trait = compute_trait_enrichment(
    mdata, 
    prog_key=prog_key,
    data_key=data_key,
    gwas_data=trait_enrichment_config['gwas_data'],
    prog_nam=trait_enrichment_config['prog_nam'],
    library=trait_enrichment_config['library'],
    n_jobs=trait_enrichment_config['n_jobs'],
    inplace=trait_enrichment_config['inplace'],
    key_column=trait_enrichment_config['key_column'],
    gene_column=trait_enrichment_config['gene_column'],
    method=trait_enrichment_config['method'],
    n_top=trait_enrichment_config['n_top'],
    low_cutoff=trait_enrichment_config['low_cutoff'],
    min_size=trait_enrichment_config['min_size'],
    max_size=trait_enrichment_config['max_size'],
)
if trait_enrichment_config["method"] == "fisher":
    pre_res_trait = pre_res_trait.rename(columns={"Term": "term", "P-value": "pval", "Adjusted P-value": "adj_pval", "Odds Ratio": "enrichment", "Genes": "genes"})
elif trait_enrichment_config["method"] == "gsea":
    pre_res_trait = pre_res_trait.rename(columns={"Term": "term", "NOM p-val": "pval", "FDR q-val": "adj_pval", "NES": "enrichment", "Lead_genes": "genes"})
data = process_enrichment_data(
    enrich_res=pre_res_trait,
    metadata=trait_enrichment_config['metadata'],
    pval_col=trait_enrichment_config["pval_col"],
    enrich_geneset_id_col=trait_enrichment_config["enrich_geneset_id_col"],
    metadata_geneset_id_col=trait_enrichment_config["metadata_geneset_id_col"],
    color_category_col=trait_enrichment_config["color_category_col"],
    program_name_col=trait_enrichment_config["program_name_col"],
    annotation_cols=trait_enrichment_config["annotation_cols"],
)
data.to_csv(os.path.join(path_out, f"{prog_key}_{trait_enrichment_config['library']}_{trait_enrichment_config['method']}_trait_enrichment.txt"), sep='\t', index=False)

Running Fisher enrichment: 100%|██████████| 300/300 [00:35<00:00,  8.53programs/s]


In [24]:
data

,term,adj_pval,trait_efos,trait_category,program_name,enrichment,trait_reported,genes,study_id,pmid,-log10(adj_pval)
0,EFO_0004875,3.705339e-09,EFO_0004875,biological process,46,3.425818,Highest math class taken (MTAG) [MTAG],PLCL2;TCF7L2;KLHL32;DIO3;RGS6;OSBPL3;EFNA5;B3G...,GCST006568,PMID:30038396,8.431172
1,EFO_0004875,5.903075e-09,EFO_0004875,biological process,9,2.932631,Highest math class taken (MTAG) [MTAG],GABRG3;RANBP17;FOXP1;TCF7L2;SLC22A23;PLCB1;BRI...,GCST006568,PMID:30038396,8.228922
2,EFO_0004875,1.291972e-08,EFO_0004875,biological process,38,2.848790,Highest math class taken (MTAG) [MTAG],FOXP1;TCF7L2;EFNA5;LSAMP;PLXDC2;ST6GAL2;TENM4;...,GCST006568,PMID:30038396,7.888747
3,EFO_0004875,4.786142e-08,EFO_0004875,biological process,54,3.018249,Highest math class taken (MTAG) [MTAG],DIO3;LSAMP;B3GLCT;PLXDC2;NALF1;DLG2;TENM4;ST6G...,GCST006568,PMID:30038396,7.320014
4,EFO_0004875,7.314192e-08,EFO_0004875,biological process,129,3.490062,Highest math class taken (MTAG) [MTAG],RANBP17;FOXP1;KLHL32;DIO3;LSAMP;B3GLCT;MITF;PR...,GCST006568,PMID:30038396,7.135834
...,...,...,...,...,...,...,...,...,...,...,...
135886,EFO_0002690,9.593476e-01,EFO_0002690,urinary system disease,34,0.489532,Systemic lupus erythematosus,ELF1,GCST011956,PMID:33272962,0.018024
135887,EFO_0002690,9.654614e-01,EFO_0002690,urinary system disease,27,0.461551,Systemic lupus erythematosus,ACAP1,GCST011956,PMID:33272962,0.015265
135888,EFO_0002690,9.662828e-01,EFO_0002690,urinary system disease,3,0.478854,Systemic lupus erythematosus,DSE,GCST011956,PMID:33272962,0.014896
135889,EFO_0002690,9.753537e-01,EFO_0002690,urinary system disease,11,0.433982,Systemic lupus erythematosus,SYNGR1,GCST011956,PMID:33272962,0.010838


In [25]:
pre_res_trait[pre_res_trait["term"] == "EFO_0004464"]

,program_name,term,pval,adj_pval,enrichment,Combined Score,genes,overlap_numerator,overlap_denominator
113,0,EFO_0004464,4.034797e-09,8.553769e-07,2.642369,51.072519,PIP4K2A;ARHGAP31;CRMP1;KAZN;LRP8;TRIM24;TTC28;...,58,593
542,1,EFO_0004464,9.175403e-09,3.706863e-06,2.620551,48.497851,ARHGAP31;CRMP1;TCF7L2;TTC28;MAPRE2;F2R;EZR;ERG...,56,593
4084,10,EFO_0004464,1.596419e-04,5.220290e-02,2.113795,18.480019,PIK3CD;CRMP1;KDM2B;LRP8;STAT1;MAPRE2;F2R;EZR;I...,36,593
40376,100,EFO_0004464,5.899149e-06,4.058614e-04,2.576462,31.022418,KAZN;THSD4;DIO3;FBXO16;TTC28;EFNA5;B3GLCT;VEPH...,35,593
40727,101,EFO_0004464,4.010141e-04,2.045172e-02,2.154842,16.854129,KAZN;THSD4;DIO3;FBXO16;EFNA5;B3GLCT;VEPH1;THBS...,30,593
...,...,...,...,...,...,...,...,...,...
38466,95,EFO_0004464,8.904229e-10,3.437032e-07,3.209381,66.881343,KAZN;THSD4;DIO3;FBXO16;TTC28;B3GLCT;F2R;FBXL7;...,46,593
38848,96,EFO_0004464,2.911182e-05,2.171741e-03,2.328892,24.323799,KAZN;TCF7L2;THSD4;DIO3;FBXO16;B3GLCT;VEPH1;EZR...,36,593
39220,97,EFO_0004464,3.156789e-05,2.329710e-03,2.383982,24.706090,KAZN;TCF7L2;THSD4;DIO3;FBXO16;EFNA5;B3GLCT;VEP...,34,593
39599,98,EFO_0004464,9.463288e-10,3.851558e-07,3.029426,62.946712,KAZN;THSD4;DIO3;GRIP1;NRCAM;TTC28;B3GLCT;SLC13...,50,593


## Run motif enrichment analysis

In [13]:
motif_enrichment_config = config['motif_enrichment']
motif_enrichment_config

{'motif_file': '/cellar/users/aklie/opt/gene_program_evaluation/src/tests/test_data/motifs.meme',
 'seq_file': '/cellar/users/aklie/data/ref/genomes/hg38/hg38.fa',
 'loci_files': ['/cellar/users/aklie/opt/gene_program_evaluation/examples/datasets/iPSC_EC/EnhancerPredictions_D0_2024_09_07.txt',
  '/cellar/users/aklie/opt/gene_program_evaluation/examples/datasets/iPSC_EC/EnhancerPredictions_sample_D1_2024_09_07.txt',
  '/cellar/users/aklie/opt/gene_program_evaluation/examples/datasets/iPSC_EC/EnhancerPredictions_sample_D2_2024_09_07.txt',
  '/cellar/users/aklie/opt/gene_program_evaluation/examples/datasets/iPSC_EC/EnhancerPredictions_sample_D3_2024_09_07.txt'],
 'names': ['D0', 'sample_D1', 'sample_D2', 'sample_D3'],
 'output_loc': None,
 'window': 1000,
 'threshold': 0.001,
 'eps': 0.001,
 'reverse_complement': True,
 'sig': 0.05,
 'num_genes': None,
 'correlation': 'pearsonr',
 'n_jobs': -1,
 'inplace': False}

In [14]:
# Run motif enrichment and save results
loci_files = motif_enrichment_config['loci_files']
names = motif_enrichment_config['names']
for loci_file, name in zip(loci_files, names):
    logging.info(f'Running motif enrichment analysis for {loci_file}')
    motif_match_df, motif_count_df, motif_enrichment_df = compute_motif_enrichment(
        mdata, 
        prog_key=prog_key,
        data_key=data_key,
        loci_file=loci_file,
        **motif_enrichment_config,
    )
    motif_match_df.to_csv(os.path.join(path_out, f'{prog_key}_enhancer_test_{motif_enrichment_config["correlation"]}_sample_{name}_motif_match.txt'), sep='\t', index=False)
    motif_count_df.to_csv(os.path.join(path_out, f'{prog_key}_enhancer_test_{motif_enrichment_config["correlation"]}_sample_{name}_motif_count.txt'), sep='\t', index=False)
    motif_enrichment_df.to_csv(os.path.join(path_out, f'{prog_key}_enhancer_test_{motif_enrichment_config["correlation"]}_sample_{name}_motif_enrichment.txt'), sep='\t', index=False)

INFO:root:Running motif enrichment analysis for /cellar/users/aklie/opt/gene_program_evaluation/examples/datasets/iPSC_EC/EnhancerPredictions_D0_2024_09_07.txt


/cellar/users/aklie/opt/miniconda3/envs/test_celloracle/lib/python3.10/site-packages/anndata/_core/anndata.py:1899: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/cellar/users/aklie/opt/miniconda3/envs/test_celloracle/lib/python3.10/site-packages/mudata/_core/mudata.py:479: UserWarning: var_names are not unique. To make them unique, call `.var_names_make_unique`.
  warnings.warn(


Number of matching genes: 4420
Number of loci: 15692
There are 54366 significant motif matches.


Computing motif enrichment:   0%|          | 0/8 [00:00<?, ?motifs/s]

INFO:root:Running motif enrichment analysis for /cellar/users/aklie/opt/gene_program_evaluation/examples/datasets/iPSC_EC/EnhancerPredictions_sample_D1_2024_09_07.txt
/cellar/users/aklie/opt/miniconda3/envs/test_celloracle/lib/python3.10/site-packages/anndata/_core/anndata.py:1899: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/cellar/users/aklie/opt/miniconda3/envs/test_celloracle/lib/python3.10/site-packages/mudata/_core/mudata.py:479: UserWarning: var_names are not unique. To make them unique, call `.var_names_make_unique`.
  warnings.warn(


...

Number of matching genes: 4420
Number of loci: 16164
There are 53283 significant motif matches.


Computing motif enrichment:   0%|          | 0/8 [00:00<?, ?motifs/s]

INFO:root:Running motif enrichment analysis for /cellar/users/aklie/opt/gene_program_evaluation/examples/datasets/iPSC_EC/EnhancerPredictions_sample_D3_2024_09_07.txt
/cellar/users/aklie/opt/miniconda3/envs/test_celloracle/lib/python3.10/site-packages/anndata/_core/anndata.py:1899: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/cellar/users/aklie/opt/miniconda3/envs/test_celloracle/lib/python3.10/site-packages/mudata/_core/mudata.py:479: UserWarning: var_names are not unique. To make them unique, call `.var_names_make_unique`.
  warnings.warn(


Number of matching genes: 4420
Number of loci: 16263
There are 50764 significant motif matches.


Computing motif enrichment:   0%|          | 0/8 [00:00<?, ?motifs/s]

## Run explained variance

In [30]:
# Run explained variance
explained_variance_config = config['explained_variance']
explained_variance_ratio = compute_explained_variance_ratio(
    mdata, 
    prog_key=prog_key,
    data_key=data_key,
    **explained_variance_config,
)
explained_variance_ratio.index = mdata.mod[prog_key].var.index
explained_variance_ratio.index.name = 'program_name'
explained_variance_ratio.columns = ["variance_explained_ratio"]
explained_variance_ratio.to_csv(os.path.join(path_out, f'{prog_key}_variance_explained_ratio.txt'), sep='\t', index=True)

Computing explained variance: 100%|██████████| 50/50 [14:40<00:00, 17.61s/programs]


## Software versions

In [17]:
# Save software versions
import joblib
import numpy as np
import scipy
import sklearn
import statsmodels
import scikit_posthocs as posthocs
import gseapy
import tangermeme

versions = {
    "evaluation_pipeline_versions": {
        'gene_program_evaluation': '0.0.1',
        'numpy': np.__version__,
        'pandas': pd.__version__,
        'mudata': mudata.__version__,
        'scipy': scipy.__version__,
        'scikit-learn': sklearn.__version__,
        'scikit-posthocs': posthocs.__version__,
        'statsmodels': statsmodels.__version__,
        'gseapy': gseapy.__version__,  # gene set enrichment analysis
        'tangermeme': tangermeme.__version__,  # motif enrichment analysis
    }
}

with open(os.path.join(path_out, 'software_versions.yml'), 'w') as f:
    yaml.dump(versions, f)

# DONE!

---